In [1]:
%pip install open_clip_torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from PIL import Image
import open_clip


#  ('EVA01-g-14', 'laion400m_s11b_b41k'),
#  ('EVA01-g-14-plus', 'merged2b_s11b_b114k'),
#  ('EVA02-B-16', 'merged2b_s8b_b131k'),
#  ('EVA02-L-14', 'merged2b_s4b_b131k'),
#  ('EVA02-L-14-336', 'merged2b_s6b_b61k'),
#  ('EVA02-E-14', 'laion2b_s4b_b115k'),
#  ('EVA02-E-14-plus', 'laion2b_s9b_b144k'),


model, _, preprocess = open_clip.create_model_and_transforms('EVA01-g-14', pretrained='laion400m_s11b_b41k')
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer('EVA01-g-14')

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
import requests
from PIL import Image
image = Image.open(requests.get(url, stream=True).raw)
import matplotlib.pyplot as plt

image = preprocess(image).unsqueeze(0)
args = ["a photo of a cat", "a photo of a dog", "an asparagus"]
text = tokenizer(args)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)
for i, arg in enumerate(args):
    prob = text_probs[0, i].item()
    print(f"{arg:<20} {prob * 100:.2f}%")

Label probs: tensor([[1.0000e+00, 1.6841e-07, 7.0158e-08]])
a photo of a cat     100.00%
a photo of a dog     0.00%
an asparagus         0.00%
